In [ ]:
# import dependencies
import pandas as pd
import json
import requests
from sqlalchemy import create_engine
from config import wlm_api, myPass
from pandas.io.json import json_normalize
import time

In [ ]:
# Walmart api key from config file
apiKey = wlm_api

In [ ]:
# api Call to walmart for the product selected 20 max on 1 call
product_api = (f"http://api.walmartlabs.com/v1/items?ids=51800359,986978174,42490113,317677075,545584365,55449983,38676400,189542267,55420123,38317400,275846958,50820749,47090515,39073013,878167&apiKey={apiKey}&lsPublisherId=").format(apiKey)

In [ ]:
# request call to APU
response = requests.get(product_api)
productList = response.json()
# convert api call to DataFrame
productList = pd.DataFrame.from_dict(productList['items'], orient='columns', dtype=None)
# Grab data desired and use as column name
productList = productList[['itemId', 'name', 'salePrice', 'clearance', 'stock']]
# display the dataframe
productList

In [ ]:
# create a list of the items for api call loop
itemId = [51800359,986978174,42490113,317677075,545584365,55449983,38676400,189542267,55420123,38317400,
              275846958,50820749,47090515,39073013,878167]
items = []
# review_product_api
for item in itemId:
    url = 'http://api.walmartlabs.com/v1/reviews/' + str(item) + '?apiKey=' + str(apiKey) + '&lsPublisherId=&format=json'
    response = requests.get(url)
    prodReview = response.json()
    time.sleep(2)
#         json.dumps(response)
    items.append(prodReview)

In [ ]:
# convert api call to DataFrame
prodReview = pd.DataFrame.from_dict(items, orient='columns', dtype=None)
# Grab data desired and use as column name
prodReview = prodReview[['itemId','reviewStatistics']]
for kwy in prodReview:
    print(kwy)

In [ ]:
reviewStats = prodReview['reviewStatistics'].apply(pd.Series)
reviewStats

In [ ]:
prodRatingOver = pd.concat([prodReview, reviewStats], axis=1)
prodRatingOver = prodRatingOver[['itemId', 'averageOverallRating', 'overallRatingRange', 'totalReviewCount']]
prodRatingOver

In [ ]:
mergedDF = pd.merge(productList, prodRatingOver, on='itemId')
mergedDF

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import mysql.connector
from mysql.connector import errorcode

# Update your password in the config file to RUN

In [ ]:
# mySql Pass word 
myPas = myPass

In [ ]:
# Create Connection to MYSQL
try:
    conn = mysql.connector.connect(
        host='localhost', user='root', password=myPas)
    mycursor = conn.cursor()
    # Error out if something happens and step out
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    # Drop table if Exists
    sqlDrop = 'DROP DATABASE IF EXISTS shopsmarter'

In [ ]:
mycursor.execute(sqlDrop)

In [ ]:
mycursor.execute('CREATE DATABASE shopsmarter')

In [ ]:
database_username = 'root'
database_password = myPas
database_ip       = '127.0.0.1'
database_name     = 'shopsmarter'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name), pool_recycle=1, pool_timeout=57600).connect()
# Create Prodcut list Table in Database from the product list dataframe
productList.to_sql(con=database_connection, name='productlist', if_exists='replace')

In [ ]:
# set couple parameters to verify data on Database
sqlQPL = 'SELECT * FROM productlist'
shDB = 'USE shopsmarter'

In [ ]:
# Set out Database 
mycursor.execute(shDB)

In [ ]:
# Select data from the productlist Table 
mycursor.execute(sqlQPL)

In [ ]:
# Verify the data set
myresult = mycursor.fetchall()

In [ ]:
# loop through the data set
for x in myresult:
  print(x)

In [ ]:
# Create product review table from the product rating dataframe
prodRatingOver.to_sql(con=database_connection, name='prodratingsoverview', if_exists='replace')

In [ ]:
# Create product review table from the product rating dataframe
mergedDF.to_sql(con=database_connection, name='combineddataframe', if_exists='replace')